In [1]:
import os
import tqdm
os.chdir("../")

In [2]:
from parser_FAR_dvf import *

In [8]:
dvf = FAR_DVF.from_file("resources/dfa_nfa_proofs.dvf", pre_scan=True)

### Regrouping all DFA with same number of states together (<= 7 states)

In [9]:
dfas_per_state_count = {}
with open("resources/dfa_nfa_proofs.dvf", "rb") as f:
    for i in tqdm.tqdm(range(dvf.n_entries)):
        header, entry = dvf.ith_entry(f, i)
        
        if entry.nb_dfa_states > 7:
            continue
        
        if entry.nb_dfa_states not in dfas_per_state_count:
            dfas_per_state_count[entry.nb_dfa_states] = {}
        
        dfa_repr = str(entry.dfa_transitions)
        if dfa_repr not in dfas_per_state_count[entry.nb_dfa_states]:
            dfas_per_state_count[entry.nb_dfa_states][dfa_repr] = 0
            
        dfas_per_state_count[entry.nb_dfa_states][dfa_repr] += 1

100%|██████████████████████████████████████████████████████████████| 534866/534866 [00:12<00:00, 42504.34it/s]


In [13]:
for i in range(1,8):
    count = 0
    for dfa in dfas_per_state_count[i]:
        count += dfas_per_state_count[i][dfa]
    print(f"{len(dfas_per_state_count[i])} DFA with {i} states solving {count} machines")

1 DFA with 1 states solving 57640 machines
3 DFA with 2 states solving 75822 machines
37 DFA with 3 states solving 102239 machines
407 DFA with 4 states solving 140760 machines
1777 DFA with 5 states solving 88778 machines
3549 DFA with 6 states solving 37930 machines
1090 DFA with 7 states solving 3181 machines


### Automata minimization
Are all the DFAs minimal?

In [21]:
from automata.fa.dfa import DFA

In [24]:
def dfa_transitions_to_automata_lib(dfa_transitions):
    states = set({})
    for i in range(len(dfa_transitions)):
        states.add(str(i))
    input_symbols={'0', '1'}
    transitions = {}
    for i in range(len(dfa_transitions)):
        transitions[str(i)] = {'0': str(dfa_transitions[i][0]),
                               '1': str(dfa_transitions[i][1])}
    initial_state = '0'
    final_states = set({})
    return DFA(states=states,input_symbols=input_symbols,transitions=transitions,
               initial_state=initial_state,
               final_states=final_states)

In [20]:

# DFA which matches all binary strings ending in an odd number of '1's
dfa = DFA(
    states={'q0', 'q1', 'q2'},
    input_symbols={'0', '1'},
    transitions={
        'q0': {'0': 'q0', '1': 'q1'},
        'q1': {'0': 'q0', '1': 'q2'},
        'q2': {'0': 'q2', '1': 'q1'}
    },
    initial_state='q0',
    final_states={'q1'}
)

In [15]:
dfas_per_state_count[2]

{'[[0, 1], [0, 1]]': 55676, '[[0, 1], [0, 0]]': 19964, '[[0, 1], [1, 1]]': 182}

In [29]:
dfa_transitions_to_automata_lib([[0, 1], [0, 1]]).minify()

DFA(states={0}, input_symbols={'0', '1'}, transitions={0: {'0': 0, '1': 0}}, initial_state=0, final_states=set(), allow_partial=False)

In [31]:
def nb_dfa_n_states_2_symbols(n):
    return n**(2*n)

In [34]:
nb_dfa_n_states_2_symbols(7)

678223072849

In [30]:
dfas_per_state_count[3]

{'[[0, 1], [2, 0], [1, 0]]': 679,
 '[[0, 1], [0, 2], [2, 2]]': 8725,
 '[[0, 1], [1, 2], [2, 2]]': 3266,
 '[[0, 1], [2, 1], [1, 1]]': 1133,
 '[[0, 1], [2, 1], [0, 1]]': 5555,
 '[[0, 1], [0, 2], [1, 2]]': 2480,
 '[[0, 1], [0, 2], [2, 1]]': 340,
 '[[0, 1], [0, 2], [0, 1]]': 10371,
 '[[0, 1], [2, 2], [0, 1]]': 6342,
 '[[0, 1], [2, 2], [2, 1]]': 620,
 '[[0, 1], [0, 2], [0, 2]]': 10270,
 '[[0, 1], [2, 1], [0, 0]]': 824,
 '[[0, 1], [2, 0], [0, 0]]': 97,
 '[[0, 1], [2, 1], [2, 2]]': 9646,
 '[[0, 1], [2, 1], [1, 0]]': 731,
 '[[0, 1], [2, 0], [0, 1]]': 2923,
 '[[0, 1], [2, 2], [0, 2]]': 117,
 '[[0, 1], [2, 0], [0, 2]]': 164,
 '[[0, 1], [2, 0], [2, 2]]': 28188,
 '[[0, 1], [0, 2], [1, 1]]': 662,
 '[[0, 1], [2, 0], [1, 1]]': 562,
 '[[0, 1], [2, 2], [1, 2]]': 40,
 '[[0, 1], [1, 2], [1, 2]]': 759,
 '[[0, 1], [2, 1], [2, 1]]': 59,
 '[[0, 1], [1, 2], [1, 1]]': 367,
 '[[0, 1], [0, 2], [0, 0]]': 6810,
 '[[0, 1], [2, 2], [0, 0]]': 93,
 '[[0, 1], [1, 2], [0, 0]]': 2,
 '[[0, 1], [2, 0], [2, 1]]': 75,
 '[[0,